In [40]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [41]:
more_features = False
use_staking = False
use_kernal_params = True
do_augment = True
fast = False
use_kernel_params_tuned = True

train_df = pd.read_csv('input/train' + ('_more_features' if more_features else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_more_features' if more_features else '') + '.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [42]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [43]:
_, X_bottomhalf, _, y_bottomhalf = train_test_split(X, y, test_size=0.8, random_state=10)

In [44]:
if use_staking:
    X = X_bottomhalf
    y = y_bottomhalf

In [45]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [46]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8990901412442585,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample': 0.8054415526396443,
    'min_child_weight': 38.138072621096654,
    'num_threads': 4,
    'max_bin': 483
}

if use_kernal_params:
    # https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment (lb: 0.901)
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 13,
        "learning_rate" : 0.01,
        "bagging_freq": 5,
        "bagging_fraction" : 0.4,
        "feature_fraction" : 0.05,
        "min_data_in_leaf": 80,
        "min_sum_heassian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

if use_kernel_params_tuned:
    # apply tuning over kernel params
    # optune score: 0.09980981822179502 
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 15,
        "learning_rate" : 0.01,
        "bagging_freq": 4,
        "bagging_fraction" : 0.4751059528094359,
        "feature_fraction" : 0.04027026162992961,
        "min_data_in_leaf": 75,
        "min_sum_heassian_in_leaf": 12,
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

In [47]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [48]:
oof = train_df[['ID_code', 'target']]
oof['predict'] = 0

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
predictions = test_df[['ID_code']]
val_aucs = []

In [50]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    N = 20
    
    if not do_augment:
        N = 1
    
    p_valid,yp = 0,0
    for i in range(N):
        if do_augment:
            X_t, y_t = augment(X_train.values, y_train.values)
            X_t = pd.DataFrame(X_t)
            X_t = X_t.add_prefix('var_')

            trn_data = lgb.Dataset(X_t, label=y_t)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        else:
            trn_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        
        evals_result = {}
        
        early_stopping_rounds = 800 if fast else 3000
        
        lgb_clf = lgb.train(params,
                        trn_data,
                        num_boost_round=2000000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval = 300,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    
    oof['predict'][valid_index] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold_n+1)] = yp/N

Fold 0 started at Mon Apr  1 08:16:51 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.882621	valid_1's auc: 0.871412
[600]	training's auc: 0.893208	valid_1's auc: 0.881989
[900]	training's auc: 0.898072	valid_1's auc: 0.886687
[1200]	training's auc: 0.900808	valid_1's auc: 0.888663
[1500]	training's auc: 0.902819	valid_1's auc: 0.89016
[1800]	training's auc: 0.904588	valid_1's auc: 0.891397
[2100]	training's auc: 0.906365	valid_1's auc: 0.892662
[2400]	training's auc: 0.908134	valid_1's auc: 0.89372
[2700]	training's auc: 0.909537	valid_1's auc: 0.894705
[3000]	training's auc: 0.910852	valid_1's auc: 0.895371
[3300]	training's auc: 0.912175	valid_1's auc: 0.896051
[3600]	training's auc: 0.913388	valid_1's auc: 0.89658
[3900]	training's auc: 0.914432	valid_1's auc: 0.897111
[4200]	training's auc: 0.91552	valid_1's auc: 0.897532
[4500]	training's auc: 0.91652	valid_1's auc: 0.89801
[4800]	training's auc: 0.917539	valid_1's auc: 0.898414
[5100]

[14400]	training's auc: 0.94145	valid_1's auc: 0.900841
[14700]	training's auc: 0.942059	valid_1's auc: 0.900795
[15000]	training's auc: 0.942668	valid_1's auc: 0.900826
[15300]	training's auc: 0.94328	valid_1's auc: 0.900772
Early stopping, best iteration is:
[12567]	training's auc: 0.93754	valid_1's auc: 0.901001
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.88352	valid_1's auc: 0.871123
[600]	training's auc: 0.893789	valid_1's auc: 0.881026
[900]	training's auc: 0.898765	valid_1's auc: 0.885737
[1200]	training's auc: 0.901538	valid_1's auc: 0.887963
[1500]	training's auc: 0.903608	valid_1's auc: 0.88952
[1800]	training's auc: 0.905384	valid_1's auc: 0.890928
[2100]	training's auc: 0.907159	valid_1's auc: 0.892299
[2400]	training's auc: 0.908876	valid_1's auc: 0.893367
[2700]	training's auc: 0.910282	valid_1's auc: 0.894302
[3000]	training's auc: 0.911604	valid_1's auc: 0.895061
[3300]	training's auc: 0.912919	valid_1's auc: 0.895816
[3600]	t

[13500]	training's auc: 0.939844	valid_1's auc: 0.900074
[13800]	training's auc: 0.94046	valid_1's auc: 0.900064
[14100]	training's auc: 0.94108	valid_1's auc: 0.900015
[14400]	training's auc: 0.941694	valid_1's auc: 0.899968
[14700]	training's auc: 0.942309	valid_1's auc: 0.899907
[15000]	training's auc: 0.942926	valid_1's auc: 0.899874
[15300]	training's auc: 0.943521	valid_1's auc: 0.899799
[15600]	training's auc: 0.944108	valid_1's auc: 0.899761
[15900]	training's auc: 0.944693	valid_1's auc: 0.899704
[16200]	training's auc: 0.945292	valid_1's auc: 0.899617
Early stopping, best iteration is:
[13234]	training's auc: 0.939281	valid_1's auc: 0.900169
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.88332	valid_1's auc: 0.871429
[600]	training's auc: 0.893845	valid_1's auc: 0.88131
[900]	training's auc: 0.898945	valid_1's auc: 0.886058
[1200]	training's auc: 0.901669	valid_1's auc: 0.888389
[1500]	training's auc: 0.90365	valid_1's auc: 0.889986
[1

[12900]	training's auc: 0.93842	valid_1's auc: 0.899994
[13200]	training's auc: 0.939055	valid_1's auc: 0.899985
[13500]	training's auc: 0.939684	valid_1's auc: 0.899975
[13800]	training's auc: 0.940319	valid_1's auc: 0.899974
[14100]	training's auc: 0.940932	valid_1's auc: 0.89997
[14400]	training's auc: 0.941545	valid_1's auc: 0.899939
[14700]	training's auc: 0.942162	valid_1's auc: 0.899922
[15000]	training's auc: 0.942768	valid_1's auc: 0.899929
[15300]	training's auc: 0.943384	valid_1's auc: 0.899968
Early stopping, best iteration is:
[12418]	training's auc: 0.937379	valid_1's auc: 0.900113
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.882684	valid_1's auc: 0.87071
[600]	training's auc: 0.89304	valid_1's auc: 0.881599
[900]	training's auc: 0.897989	valid_1's auc: 0.886306
[1200]	training's auc: 0.900786	valid_1's auc: 0.888443
[1500]	training's auc: 0.902768	valid_1's auc: 0.889838
[1800]	training's auc: 0.904552	valid_1's auc: 0.891212
[2

[11100]	training's auc: 0.933883	valid_1's auc: 0.900525
[11400]	training's auc: 0.934552	valid_1's auc: 0.900529
[11700]	training's auc: 0.935213	valid_1's auc: 0.900524
[12000]	training's auc: 0.935901	valid_1's auc: 0.900516
[12300]	training's auc: 0.93655	valid_1's auc: 0.900486
Early stopping, best iteration is:
[9569]	training's auc: 0.930363	valid_1's auc: 0.900626
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.883209	valid_1's auc: 0.871013
[600]	training's auc: 0.893599	valid_1's auc: 0.881219
[900]	training's auc: 0.898434	valid_1's auc: 0.885746
[1200]	training's auc: 0.901255	valid_1's auc: 0.887992
[1500]	training's auc: 0.903324	valid_1's auc: 0.889629
[1800]	training's auc: 0.905081	valid_1's auc: 0.891052
[2100]	training's auc: 0.906762	valid_1's auc: 0.892332
[2400]	training's auc: 0.908465	valid_1's auc: 0.893418
[2700]	training's auc: 0.909856	valid_1's auc: 0.894383
[3000]	training's auc: 0.911169	valid_1's auc: 0.89512
[3300

...

[3600]	training's auc: 0.91469	valid_1's auc: 0.892772
[3900]	training's auc: 0.915762	valid_1's auc: 0.893477
[4200]	training's auc: 0.916846	valid_1's auc: 0.894006
[4500]	training's auc: 0.917907	valid_1's auc: 0.894489
[4800]	training's auc: 0.918917	valid_1's auc: 0.894974
[5100]	training's auc: 0.919873	valid_1's auc: 0.895307
[5400]	training's auc: 0.920816	valid_1's auc: 0.895633
[5700]	training's auc: 0.921704	valid_1's auc: 0.895838
[6000]	training's auc: 0.922588	valid_1's auc: 0.896068
[6300]	training's auc: 0.923432	valid_1's auc: 0.896249
[6600]	training's auc: 0.924253	valid_1's auc: 0.896463
[6900]	training's auc: 0.925071	valid_1's auc: 0.896623
[7200]	training's auc: 0.925851	valid_1's auc: 0.896715
[7500]	training's auc: 0.926621	valid_1's auc: 0.896829
[7800]	training's auc: 0.927387	valid_1's auc: 0.896863
[8100]	training's auc: 0.928151	valid_1's auc: 0.896968
[8400]	training's auc: 0.928909	valid_1's auc: 0.897041
[8700]	training's auc: 0.929627	valid_1's auc: 0.

[4800]	training's auc: 0.917903	valid_1's auc: 0.895185
[5100]	training's auc: 0.918846	valid_1's auc: 0.895571
[5400]	training's auc: 0.919786	valid_1's auc: 0.895886
[5700]	training's auc: 0.920675	valid_1's auc: 0.896126
[6000]	training's auc: 0.921565	valid_1's auc: 0.896432
[6300]	training's auc: 0.922434	valid_1's auc: 0.896615
[6600]	training's auc: 0.923272	valid_1's auc: 0.896734
[6900]	training's auc: 0.924093	valid_1's auc: 0.896896
[7200]	training's auc: 0.924873	valid_1's auc: 0.89701
[7500]	training's auc: 0.92564	valid_1's auc: 0.897155
[7800]	training's auc: 0.926394	valid_1's auc: 0.8972
[8100]	training's auc: 0.927156	valid_1's auc: 0.897246
[8400]	training's auc: 0.927906	valid_1's auc: 0.897311
[8700]	training's auc: 0.928636	valid_1's auc: 0.897338
[9000]	training's auc: 0.929357	valid_1's auc: 0.897391
[9300]	training's auc: 0.93007	valid_1's auc: 0.897412
[9600]	training's auc: 0.93078	valid_1's auc: 0.897447
[9900]	training's auc: 0.931478	valid_1's auc: 0.89744

[8100]	training's auc: 0.926574	valid_1's auc: 0.897087
[8400]	training's auc: 0.927327	valid_1's auc: 0.897162
[8700]	training's auc: 0.928073	valid_1's auc: 0.897192
[9000]	training's auc: 0.928799	valid_1's auc: 0.897293
[9300]	training's auc: 0.929502	valid_1's auc: 0.897312
[9600]	training's auc: 0.930218	valid_1's auc: 0.897334
[9900]	training's auc: 0.93093	valid_1's auc: 0.897363
[10200]	training's auc: 0.931647	valid_1's auc: 0.897387
[10500]	training's auc: 0.932331	valid_1's auc: 0.897346
[10800]	training's auc: 0.933018	valid_1's auc: 0.897319
[11100]	training's auc: 0.933704	valid_1's auc: 0.897293
[11400]	training's auc: 0.934377	valid_1's auc: 0.897302
[11700]	training's auc: 0.935046	valid_1's auc: 0.897323
[12000]	training's auc: 0.935712	valid_1's auc: 0.897253
[12300]	training's auc: 0.936378	valid_1's auc: 0.897288
[12600]	training's auc: 0.937022	valid_1's auc: 0.897342
[12900]	training's auc: 0.937672	valid_1's auc: 0.897283
Early stopping, best iteration is:
[101

[10200]	training's auc: 0.932486	valid_1's auc: 0.897371
[10500]	training's auc: 0.933167	valid_1's auc: 0.897375
[10800]	training's auc: 0.933843	valid_1's auc: 0.897311
[11100]	training's auc: 0.934524	valid_1's auc: 0.897293
[11400]	training's auc: 0.935196	valid_1's auc: 0.897248
[11700]	training's auc: 0.935855	valid_1's auc: 0.897258
[12000]	training's auc: 0.936518	valid_1's auc: 0.897214
[12300]	training's auc: 0.937181	valid_1's auc: 0.897157
[12600]	training's auc: 0.937828	valid_1's auc: 0.897147
[12900]	training's auc: 0.938477	valid_1's auc: 0.89714
Early stopping, best iteration is:
[10077]	training's auc: 0.932209	valid_1's auc: 0.897415
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.883143	valid_1's auc: 0.868263
[600]	training's auc: 0.893528	valid_1's auc: 0.877782
[900]	training's auc: 0.898465	valid_1's auc: 0.881389
[1200]	training's auc: 0.901372	valid_1's auc: 0.883897
[1500]	training's auc: 0.90341	valid_1's auc: 0.885751

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

In [51]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.901727883, std: 0.003113202. All auc: 0.901708018.


In [52]:
# submission
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("lgb_submission.csv", index=False)
oof.to_csv('lgb_oof.csv', index=False)

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
